In [1]:
# create feature pickles to use around the place
# humpback whale pickle with new annots
import sys

sys.path.append("/Users/david/Documents/mastersCode/ubm")
print(sys.path)

['/Users/david/Documents/mastersCode/ubm/final/tests/get_features_humpback', '/Users/david/.conda/envs/ubm/lib/python310.zip', '/Users/david/.conda/envs/ubm/lib/python3.10', '/Users/david/.conda/envs/ubm/lib/python3.10/lib-dynload', '', '/Users/david/.conda/envs/ubm/lib/python3.10/site-packages', '/Users/david/Documents/mastersCode/ubm']


In [2]:
import utils
import matplotlib.pyplot as plt
import numpy as np

In [29]:
import librosa

def get_feature(samp, nfft, fs=4000):
    # add optionals here if needed
    # mfcc
    feat = librosa.feature.mfcc(y=samp,sr=fs, n_mfcc=4, n_fft=nfft,hop_length=int(nfft/4)).T
    # feat = useful.get_log_power_feature(samp, nfft=nfft)[2:,:]
    return feat

def normalize_features(feats, per_feature=False):
    if per_feature:
        normalised_feats = []
        for feat in feats:
            mean = np.mean(feat, axis=0)
            std_dev = np.std(feat, axis=0)
            normalised_feats.append((feat - mean) / std_dev)
        return normalised_feats
    else:
        all_features_concat = np.concatenate(feats)
        mean = np.mean(all_features_concat, axis=0)
        std_dev = np.std(all_features_concat, axis=0)
        return [((feat - mean) / std_dev) for feat in feats], {'mean': mean, 'std': std_dev}

def average_features(some_feature, avg_over, d):
    new_n = len(some_feature) // avg_over
    avg_features = np.empty((new_n, d))

    for i in range(new_n):
        start_idx = i * avg_over
        end_idx = (i + 1) * avg_over
        avg_features[i, :] = np.mean(some_feature[start_idx:end_idx, :], axis=0)
    return avg_features

In [30]:
from whale.setup import get_data, constants, annotations
from whale.setup.constants_01 import *
import importlib
importlib.reload(constants)

importlib.reload(get_data)
from final import useful
importlib.reload(useful)

# constants for fe and what not
fs = 4000 #4000 49000

labels_set = ['MOO', 'HIGH_MOO', 'LOW_MOO', 'PULSE', 'SWOOP', 'NOISE']

num_to_label = {i: labels_set[i] for i in range(len(labels_set))}
label_to_num = {labels_set[i] : i for i in range(len(labels_set))}

get_data_dict = {
    'MOO': [get_data.GetDataSimple(HPB_MBY_DATA,HPB_MBY_LABEL_MOO, fs, useful.file_to_audio)],
    'HIGH_MOO': [get_data.GetDataSimple(HPB_MBY_DATA,HPB_MBY_LABEL_HIGH_MOO, fs, useful.file_to_audio)],
    'LOW_MOO': [get_data.GetDataSimple(HPB_MBY_DATA,HPB_MBY_LABEL_LOW_MOO, fs, useful.file_to_audio)],
    'PULSE': [get_data.GetDataSimple(HPB_MBY_DATA,HPB_MBY_LABEL_PULSE, fs, useful.file_to_audio)],
    'SWOOP': [get_data.GetDataSimple(HPB_MBY_DATA,HPB_MBY_LABEL_SWOOP, fs, useful.file_to_audio)],
    'NOISE': [get_data.GetDataSimple(HPB_MBY_DATA,HPB_MBY_NOISE, fs, useful.file_to_audio)]
}

In [31]:
samples, labels = useful.get_samples(get_data_dict, labels_set, label_to_num)

__collected samples__
MOO :  23
HIGH_MOO :  15
LOW_MOO :  28
PULSE :  24
SWOOP :  44
NOISE :  24


In [32]:
# from scipy.interpolate import interp1d
#
all_features = []
nfft = 128 # 256 # 256 # 256 #128 # 256 #128
to_remove = []

for i in range(len(samples)):
    sample = samples[i]
    feat_b = get_feature(sample, nfft)
    all_features.append(feat_b) #152 129

/Users/david/.conda/envs/ubm/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


In [33]:
np.min([len(feat) for feat in all_features])

54

In [34]:
normalized_features, mean_std = normalize_features(all_features, False)
averaged_features = [average_features(feat, 3, feat.shape[1]) for feat in all_features]
averaged_normalised_features = [average_features(feat, 3, feat.shape[1]) for feat in normalized_features]

In [38]:
import final.cross_validation as cv
from final.cross_validation import *
import importlib
importlib.reload(cv)
from hmmlearn.hmm import GaussianHMM
n_folds = 2
# cv_output = cv.split_data_for_cross_validation(samples, labels, n_folds, test_ratio=0.2)
cv_output = cv.split_data_for_cross_validation(normalized_features, labels, n_folds, test_ratio=0.2)

In [39]:
import pickle
# Pickle for reading in Python
output = open('cv_output_hpb_mfcc_norm' + ".pkl", "wb")
pickle.dump({'cv_output': cv_output, 'label_map': label_to_num, 'num_map': num_to_label}, output)
output.close()